# 生命游戏

+ 每个网格称为cell
+ 每个cell有两种状态，alive和dead，分别用两种不同的颜色来表示
+ 每个cell的状态变化受其周围8个cell影响
+ 如果一个活的(alive)cell，其周围的活的cell数量少于2——即为0或1，则这个cell将死去(dead)
+ 如果一个活的cell，其周围的活的cell数量为2或3，则它继续存活
+ 如果一个活的cell，其周围的活的cell数量超过3——即为4至8，则这个cell将死去
+ 如果一个死的cell，其周围的活的cell数量为3，则它成为活的cell，否则仍然为死的cell

### 前提知识点
> IPython.display
 from IPython.display import clear_output, display_html
 
     display_html(*objs, **kwargs)
- 参数一：数据
- 参数二：是否在演示前格式化，默认false

    clear_output() 
- 每一次演示后会清空画布

> enumerate()   类似range
  
>定义:对于一个可迭代的（iterable）/可遍历的对象（如列表、字符串），enumerate将其组成一个索引序列，利用它可以同时获得索引和值

- 如果对一个列表，既要遍历索引又要遍历元素时，首先可以这样写：
      list1 = ["这", "是", "一个", "测试"]
      for i in range (len(list1)):
          print i ,list1[i]  
- 上述方法有些累赘，利用enumerate()会更加直接和优美：
      list1 = ["这", "是", "一个", "测试"]
      for index, item in enumerate(list1):
          print index, item
      0 这
      1 是
      2 一个
      3 测试 
- enumerate还可以接收第二个参数，用于指定索引起始值，如：
      list1 = ["这", "是", "一个", "测试"]
      for index, item in enumerate(list1, 1):  ##指定序号从1开始
      print index, item
      1 这
      2 是
      3 一个
      4 测试

#### 以坐标定义初始世界

In [14]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
from collections import Counter  ##统计

def neighbors(cell):
    (x,y) = cell
    return [ (x-1, y-1),(x, y-1),(x+1,y-1),
             (x-1, y),           (x+1,y),
             (x-1,y+1), (x,y+1), (x+1,y+1)]

def neighbor_counts(world):
    """ A {cell: int} counter of number of live neighours for each cell 
    """
    return Counter(nb for cell in world      ##统计一下，嵌套循环，从中计算细胞中，检查有多少或者的邻居
                   for nb in neighbors(cell))
    
def next_generation(world):  ##下一代的结果
    possible_cells = counts = neighbor_counts(world)  ##对于邻居做一次count
    return { cell for cell in possible_cells
            if (counts[cell] == 3)
            or (counts[cell] == 2 and cell in world)}

In [3]:
import time
from IPython.display import clear_output, display_html  ##display 可以消除output，不会把之前的输出留下

LIVE = '@'  ##活的
EMPTY = '.'  ##死的
PAD =' '

def display_run(world, n=10, Xs=range(10), Ys=range(10), pause=0.5):  ##停顿0.5s
    for g in range(n+1):##n是演化的次数
        clear_output()
        display_html('Generation {}, Population {} \n{}'  ##可以看到它演化到第几代
                     .format(g, len(world),pre(picture(world, Xs, Ys))), raw=True)
        time.sleep(pause) ##停几秒种
        world = next_generation(world)
        

def pre(text): return '<pre>' + text + '</pre>'

def picture(world, Xs, Ys):    ##画图
    def row(y):   ##一行一行的链接
        return PAD.join(LIVE if(x,y) in world else EMPTY for x in Xs)  ##空格画
    return '\n'.join(row(y) for y in Ys)  ##把每行join到一块

#### 根据图片得出坐标，并对初始坐标进行偏移

In [27]:
def shape(picture, offset=(3, 3)):
    "Convert a graphical picture (e.g. '@ @ .\n. @@') into a world (set of cells)."
    cells = {(x, y) 
             for (y, row) in enumerate(picture.splitlines())  ##先将picture按行拆分，并区分。row是数组,是第一排
             for (x, c) in enumerate(row.replace(PAD, ''))  ##遍历row里面的元素（出去空格部分），c就是row的元素
             if c == LIVE}
    return move(cells, offset)

def move(cells, offset):
    "Move/Translate/slide a set of cells by a (dx, dy) displacement/offset."
    (dx, dy) = offset
    return {(x+dx, y+dy) for (x, y) in cells}   ##算偏移,将生成的初始点偏移

blinker     = shape("@@@")
block       = shape("@@\n@@")
beacon      = block | move(block, (2, 2))
toad        = shape(".@@@\n@@@.")
glider      = shape(".@.\n..@\n@@@")
rpentomino  = shape(".@@\n@@.\n.@.", (36, 20))
line        = shape(".@@@@@@@@.@@@@@...@@@......@@@@@@@.@@@@@", (10, 10))
growth      = shape("@@@.@\n@\n...@@\n.@@.@\n@.@.@", (10, 10))
    

In [28]:
display_run(rpentomino, 130,range(55), range(40))

Generation 130, Population 178 
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . . . . . . . . . . @ . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . @ . . . . . . . . . . . . . @ . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . . . . . .
. . . . . . . . . . . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . @ @ @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . . . .
. . . . . @ . @ @ @ @ . @ @ . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . . . . @ . @ @ . . . . . .
. . . @ @ . . @ @ . . @ @ @ . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . . . . @ . @ . . . @ @ . .
. . . . . @ @ @ . . . @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . . . . . . .
. . . . . . . . . . . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . @ . @ . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . . . . . . . . . . . . . . . . .
. . . . . . . . @ @ . . . . . . . . . . . . . . . . . @ . @ . . . @ @ . . . . . . . . . . . . . . . . . . . .
. . . . @ @ @ . . @ @ . . . . . . . . . . . . . . . . @ . @ . . . . . . . . . . . . . . . . @ @ . . . . . . .
. . . . @ @ . . . @ @ . . . . . . . . . . . . . . . . @ . . . . . . . . . . . . . . . . . @ @ @ . . @ @ . . .
. . . . @ . . . . . . . . . . . . . . . . . . . . . . . @ . . . . . . . . @ . . . . . . . @ . . @ . @ @ @ . .
. . . . @ @ . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . @ . . . . . . . @ . . . @ . @ . .
. . . . . @ @ @ . . . . . . . . . . . . . . . . . . . . . . . . @ . . . . . . @ . . . . . . . . . . . @ . . .
. . . . @ . . @ . . . . . . . . . . . . . . . . . . . . . . . @ . . . . . . @ . . . @ @ . . . . . . . . . . .
. . . . . @ . . . . . . . . . . . . . . . . . . . . . . . . @ . . . . . @ @ . . . @ . . @ . . . . . . . . . .
@ @ . @ @ @ . . . . . . . . . . . . . . . . . . . . . . . . @ . . @ . . . . . . . @ . . @ . . . . . . . . . .
. . @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . @ @ @ . . . . . . @ @ . . . . . . . . . . .
. . . @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . . @ . . . . . . . . . . . . . . . . . .
@ @ . @ @ . . . . . . . . . . @ @ . . . . . . . . . . . . . . . @ . . . . . . . . . . . . . . . . . . @ . . .
@ . . @ . . . . . . . . . . . @ @ . . . . . . . . . . . . . . . . . . @ . . . . . . . . . . . . . . . . @ @ .
@ @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . @ . @ . @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . @ . @ . @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . @ .

In [29]:
zoo = (move(blinker, (5, 25)) | move(glider, (8, 13)) | move(blinker, (20, 25))  |
       move(beacon, (24, 25)) | move(toad, (30, 25))  | move(block, (13, 25)) | move(block, (17, 33)))

display_run(zoo, 160, range(50), range(40))

Generation 160, Population 111 
 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . . . @ . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . . @ . @ . . @ . . @
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . @ . . @ . @ @ . @ . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . @ @ . . @ @ . . . @ . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . . @ . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ @ @ . @ @ @ @ @ . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . @ @ @ . . . @ . . . @ @ . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . . @ . @ . @ . . . . . . . . . . @
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . . . . @ . . . . . . . . @ .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . @ @ . . @ . . . . . . . @ @ .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . @ . . . . . . @ @ . @ .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . @ . . . . . . . . @ @ . @ @
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ @ . . . . . . . . @ @ . @ @
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ . . . . . . . . . . . @ @ .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . @ @ @ . . . . . @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . @ . . @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . @ @ . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . @ @ . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .